In [1]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import numpy as np

remote = mysql.connector.connect(
    host = 'database-1.cneecum0ieie.ap-northeast-2.rds.amazonaws.com',
    user = 'eda',
    password = 'eda',
    database = 'eda'
)

cursor = remote.cursor()
table_name = 'population'
query = 'SELECT * FROM {}'.format(table_name)
cursor.execute(query)
result = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
df_pop = pd.DataFrame(result, columns=columns)

cursor.close()

remote.close

<bound method CMySQLConnection.close of <mysql.connector.connection_cext.CMySQLConnection object at 0x78a58c405150>>

In [2]:
df_pop = df_pop.drop(columns=['pop_0_9','pop_10_19','pop_20_29','pop_30_39','pop_40_49', 
                     'pop_50_59', 'pop_60_69', 'pop_70_79', 'pop_80_89', 'pop_90_99', 
                     'pop_100_'], axis = 1)
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510 entries, 0 to 3509
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city     3510 non-null   object
 1   year     3510 non-null   object
 2   month    3510 non-null   object
 3   pop_sum  3510 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 109.8+ KB


In [3]:
df_pop1 = df_pop.loc[df_pop['month'] == '12']
df_pop1.head()

,city,year,month,pop_sum
187,전국,2008,12,49540367
188,서울특별시,2008,12,10200827
189,부산광역시,2008,12,3564577
190,대구광역시,2008,12,2492724
191,인천광역시,2008,12,2692696


In [4]:
df_pop1.drop(columns=['month'], inplace=True)

/tmp/ipykernel_15765/3385416431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop1.drop(columns=['month'], inplace=True)


In [5]:
df_pop2 = df_pop1.set_index(['city', 'year'])
df_pop2 = df_pop2.sort_index(level=['city', 'year'], ascending=[True, True])
df_pop2.head(20)

pop_sum
city year          
강원도  2008   1508575
     2009   1512870
     2010   1529818
     2011   1536448
     2012   1538630
     2013   1542263
     2014   1544442
     2015   1549507
     2016   1550806
     2017   1550142
     2018   1543052
     2019   1541502
     2020   1542840
     2021   1538492
     2022   1536498
     2023   1527807
경기도  2008  11292264
     2009  11460610
     2010  11786622
     2011  11937415

In [6]:
city_list = list(df_pop2.index.get_level_values("city").unique())
year_list = list((df_pop2.index.get_level_values("year")).unique())
year_list

['2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023']

In [7]:
df_pop2.loc[('전국', '2008'),'pop_sum']

49540367

In [8]:
df_pop2['inc_rate'] = 0.0

for c in city_list:
    for y in year_list[1:]:
        if (c,str(int(y)-1)) in df_pop2.index:
            df_pop2.loc[(c,y), "inc_rate"] = df_pop2.loc[(c,y),"pop_sum"]/df_pop2.loc[(c,str(int(y)-1)),"pop_sum"] - 1

df_pop2 = df_pop2.replace(0, np.nan)
df_pop2 = df_pop2.drop(columns = ['pop_sum'])

df_pop2

inc_rate
city year          
강원도  2008       NaN
     2009  0.002847
     2010  0.011203
     2011  0.004334
     2012  0.001420
...             ...
충청북도 2019  0.000472
     2020  0.000519
     2021 -0.002130
     2022 -0.001483
     2023 -0.000996

[284 rows x 1 columns]

In [9]:
# df_pop2.to_csv("~/Downloads/pop_rate.csv")